/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
a.H@  bay.H  galilee.H	inv.T


In [ ]:
import pyOperator
import giee
import Weight
import pyProblem
import numpy as np
class simpleChain(pyOperator.Operator):
    """ d = Op1 op2 m"""
    def __init__(self,op1,op2):
        self._op1=op1
        self._op2=op2
        if not self._op1.getDomain().checkSame(self._op2.getRange()):
            raise Exception("domain of op1 must be the same space range of op 2")
        super().setDomainRange(self._op2.getDomain(),self._op1.getRange())
        self._tmp=self._op1.getDomain().clone()
    
    def forward(self,add,mod,dat):
        self._op2.forward(False,mod,self._tmp)
        self._op1.forward(add,self._tmp,dat)
    def adjoint(self,add,mod,dat):
        self._op1.adjoint(False,self._tmp,dat)
        self._op2.adjoint(add,mod,self._tmp)
    

        


class missingDataProblem(pyProblem.ProblemL2Linear):
    def __init__(self,mod,op,notKnown):
        model=mod.clone()
        self.opJ=Weight.weight(notKnown,model,model)
        self.both=simpleChain(op,self.opJ)
        self.dtmp=op.range.clone()
        op.forward(False,model,self.dtmp)
        super().__init__(model,self.dtmp,self.both)
        self.opNoJ=op
    def resf(self, model):
        self.res.scaleAdd(self.dtmp, 0., 1.)
        return self.res
    

def readGocad(fle):
    f=open(fle)
    lines=f.readlines()
    x=[]
    y=[]
    z=[]
    for line in lines:
        arr=line.split()
        x.append(float(arr[2]))
        y.append(float(arr[3]))
        z.append(float(arr[4]))
    return np.asarray(x),np.asarray(y),np.asarray(z)

seisx,seisy,seisz=readGocad("../../data/s2000.5")
wellx,welly,wellz=readGocad("../../data/w2000.5")

In [ ]:
import Int2
import Hypercube
import Grey
import numpy as np
seisModel=giee.vector(Hypercube.hypercube(ns=[40,40],os=[5300,-22000],ds=[600,800]))
seisData=giee.vector(Hypercube.hypercube(ns=[len(seisz)]),arr=seisz)
wellData=giee.vector(Hypercube.hypercube(ns=[len(wellz)]),arr=wellz)
wellModel=giee.vector(Hypercube.hypercube(ns=[40,40],os=[5300,-22000],ds=[600,800]))

op=Int2.nearest(seisModel,seisData,seisx,seisy)
op2=Int2.nearest(wellModel,wellData,wellx,welly)
op.adjoint(False,seisModel,seisData)
op2.adjoint(False,wellModel,wellData)

Grey.plot(seisModel).image()+Grey.plot(wellModel).image()

In [ ]:
from numba import njit
@njit()
def averedge(x,y,z,o1,d1,o2,d2,grid):
    x1=o1
    x2=o1+d1*(grid.shape[1]-1)
    for i2 in range(grid.shape[0]):
        yy=o2+d2*i2
        wts1=0
        tot1=0
        wts2=0
        tot2=0
        for i in range(grid.shape[1]):
            wt1=(x1-x[i])*(x1-x[i])+(yy-y[i])*(yy-y[i])
            wt2=(x2-x[i])*(x2-x[i])+(yy-y[i])*(yy-y[i])
            tot1+=wt1*wt1*z[i]
            wts1+=wt1*wt1
            tot2+=wt2*wt2*z[i]
            wts2+=wt2*wt2
        if grid[i2,0] > -1:
            grid[i2,0]=tot1/wts1
        if grid[i2,grid.shape[1]-1]>-1:
            grid[i2,grid.shape[1]-1]=tot2/wts2
    y1=o2
    y2=o2+d2*(grid.shape[0]-1)
    for i1 in range(1,grid.shape[1]-1):
        xx=o1+d1*i1
        wts1=0
        tot1=0
        wts2=0
        tot2=0
        for i in range(x.shape[0]):
            wt1=(xx-x[i])*(xx-x[i])+(y1-y[i])*(y1-y[i])
            wt2=(xx-x[i])*(xx-x[i])+(y2-y[i])*(y2-y[i])
            tot1+=wt1*wt1*z[i]
            wts1+=wt1*wt1
            tot2+=wt2*wt2*z[i]
            wts2+=wt2*wt2
        if grid[0,i1] > -1:
            grid[0,i1]=tot1/wts1
        if grid[grid.shape[0]-1,i1]>-1:
            grid[grid.shape[0]-1,i1]=tot2/wts2     


In [ ]:
import copy
seisA=copy.deepcopy(seisModel.getHyper().axes)
averedge(seisx,seisy,seisz,seisA[0].o,seisA[0].d,seisA[1].o,seisA[1].d,seisModel.getNdArray())
Grey.plot(seisModel,eclip=-500).image()

In [2]:
def nonZero(field):
    out=field.clone()
    out.set(0)
    o=out.getNdArray()
    i=field.getNdArray()
    for i2 in range(i.shape[0]):
        for i1 in range(i.shape[1]):
            if i[i2,i1] > -1:
                o[i2,i1]=1
    return out

nz=nonZero(seisModel)



NameError: name 'seisModel' is not defined

In [6]:
import Igrad
import copy
import Hypercube
import giee
class igradSq(pyOperator.Operator):
    def __init__(self,mod):
        axs=copy.deepcopy(mod.getHyper().axes)
        axs.append(Hypercube.axis(n=2))
        self.dtmp=giee.vector(Hypercube.hypercube(axes=axs))
        super().__init__(mod,mod)
        self.ig=Igrad.igrad2(mod,self.dtmp)
    def forward(self,add,model,data):
        self.ig.forward(False,model,self.dtmp)
        self.ig.adjoint(add,data,self.dtmp)
    def adjoint(self,add,model,data):
        self.ig.forward(False,data  ,self.dtmp)
        self.ig.adjoint(add, model,self.dtmp)

In [7]:
import pyStopper
import pyLinearSolver
op1=igradSq(seisModel)
problem=missingDataProblem(seisModel,op1,nz)
problemStop=pyStopper.BasicStopper(niter=100)
solve=pyLinearSolver.LCGsolver(problemStop)
solve.setDefaults(save_model=True)

solve.run(problem,verbose=False)

In [8]:
Grey.plot(problem.model,colorbar=True,width=500,eclip=-500).image()

OPTS {'colorbar': True, 'width': 500, 'eclip': -500, 'invert_yaxis': True, 'invert_xaxis': False, 'transp': True, 'label1': '', 'label2': '', 'xlabel': '', 'ylabel': ''}


:Image   [x,y]   (z)

In [9]:
Grey.plot(solve.modelSet.vecSet[10]).image()+Grey.plot(solve.modelSet.vecSet[60]).image()

OPTS {'invert_yaxis': True, 'invert_xaxis': False, 'transp': True, 'label1': '', 'label2': '', 'xlabel': '', 'ylabel': ''}
OPTS {'invert_yaxis': True, 'invert_xaxis': False, 'transp': True, 'label1': '', 'label2': '', 'xlabel': '', 'ylabel': ''}


:Layout
   .Image.I  :Image   [x,y]   (z)
   .Image.II :Image   [x,y]   (z)

In [14]:
import Contour
wellMSeis=wellModel.clone()
wellMSeis.scaleAdd(problem.model,1,-1)
wellNo=nonZero(wellModel)
problem2=missingDataProblem(wellMSeis,op1,wellNo)

problemStop=pyStopper.BasicStopper(niter=500)

solve2=pyLinearSolver.LCGsolver(problemStop)
solve2.run(problem2,verbose=False)

newM=problem2.model.clone()
newM.scaleAdd(problem.model)
Grey.plot(newM,cmap="rainbow",width=600,colorbar=True).image()

OPTS {'cmap': 'rainbow', 'width': 600, 'colorbar': True, 'invert_yaxis': True, 'invert_xaxis': False, 'transp': True, 'label1': '', 'label2': '', 'xlabel': '', 'ylabel': ''}


:Image   [x,y]   (z)

In [13]:
diff=newM.clone()
diff.scaleAdd(problem.model,1,-1)
Grey.plot(diff).image()

OPTS {'invert_yaxis': True, 'invert_xaxis': False, 'transp': True, 'label1': '', 'label2': '', 'xlabel': '', 'ylabel': ''}


:Image   [x,y]   (z)